In [14]:
import pandas as pd
IMAGE_PATH = "./data_source/images/"
TEST_PATH = "./data_source/test.csv"
test_data = pd.read_csv(TEST_PATH)
SUB_PATH = "./data_source/sample_submission.csv"
sub = pd.read_csv(SUB_PATH)

In [69]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from efficientnet.keras import EfficientNetB6
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121,DenseNet201
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

img_size = 299

def get_model(model_name) :
    if model_name == 'MobileNet' :
        base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'VGG16' : 
        base_model=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'DenseNet' :
        base_model = DenseNet121(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'DenseNet201' :
        base_model = DenseNet201(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'Inception' :
        base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'ResNet' :
        base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
    elif model_name == 'EfficientNetB61' or model_name == 'EfficientNetB62' :
        base_model = EfficientNetB6(weights="imagenet",include_top=False,input_shape=(img_size,img_size,3))
        model = keras.Sequential([base_model,keras.layers.GlobalAveragePooling2D(),keras.layers.Dense(4, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
        return model
    elif model_name == 'BinaryModelB6':
        base_model = EfficientNetB6(input_shape=(img_size,img_size,3), include_top=False, weights="imagenet")
        model = keras.Sequential([base_model,keras.layers.GlobalAveragePooling2D(),keras.layers.Dense(1, activation='sigmoid')])
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
        return model
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(512,activation='relu')(x)
    x=Dropout(0.3)(x)
    x=Dense(256,activation='relu')(x) #dense layer 2
    preds=Dense(4,activation='softmax')(x)
    model=Model(inputs=base_model.input,outputs=preds,name=model_name)
    model.compile(optimizer='Adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
    return model

In [3]:
#nets = ['MobileNet','DenseNet','Inception','ResNet','EfficientNetB61','EfficientNetB62']
#bin_nets = ['BinaryModelB6']
nets = ['MobileNet','EfficientNetB61','EfficientNetB62','Inception','ResNet','DenseNet201']
bin_nets = ['BinaryModelB6']
CHECKPOINTS_BASE = 'checkpoints'
weights = {
    'MobileNet': '%s/MobileNet/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'DenseNet': '%s/DenseNet/best_model_val_loss_0.18127.h5'%CHECKPOINTS_BASE,
    'DenseNet201': '%s/DenseNet201/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'Inception': '%s/Inception/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'ResNet': '%s/ResNet/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'EfficientNetB61': '%s/effnet/best_model.hdf5'%CHECKPOINTS_BASE,
    'EfficientNetB62': '%s/effnet/best_model_1.8477e-06.hdf5'%CHECKPOINTS_BASE,
    'BinaryModelB6': 'two_classes/zweiter/best_model_f1.hdf5'}

models = []
for net in nets:
    model = get_model(net)
    model.load_weights(weights.get(net))
    models.append(model)

bin_models = []
bin_models.append(get_model(bin_nets[0]))
bin_models[0].load_weights(weights.get(bin_nets[0]))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\Users\Luca\Anaconda3\envs\tf_gpu\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [4]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from matplotlib import image
from numpy import asarray
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


X_train = []
Y_train = []

size = (299,299)
class_map = {'healthy':0,'multiple_diseases':1,'rust':2,'scab':3}

def load_image(image_id,size):  
    file_path = image_id
    with Image.open(file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

import glob
for img_path in glob.glob("data/Test/*/*.jpg"):
    Y_train.append(class_map.get(img_path.split('\\')[1]))
    X_train.append(load_image(img_path,size) / 255.)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [5]:
def average_preds(p,indices):
    preds = models[0].predict(p)
    preds.fill(0)
    for i in range(len(models)):
        if i in indices:
            preds = preds + models[i].predict(p)
    preds = preds / len(indices)
    preds_classes = []
    for pred in preds:
        preds_classes.append(np.argmax(pred))
    return preds_classes

In [6]:
from itertools import combinations
from IPython.display import clear_output

network_indices = [0,1,2,3,4,5]
ensemble_scores = {}

counter = 0
end = 63
for k in range(1,len(network_indices)+1):
    combs = list(combinations(network_indices,k))
    for comb in combs:
        preds = average_preds(X_train,list(comb))
        error = sum(preds != Y_train)
        ensemble_scores[comb] = error
        counter+=1
        print('%.2f'%((counter/end)))
        clear_output(wait=True)

1.00


In [8]:
ensemble_scores_sorted = ensemble_scores

{k: v for k, v in sorted(ensemble_scores_sorted.items(), key=lambda item: item[1])}

{(0, 1): 5,
 (1, 2): 6,
 (2,): 7,
 (0, 4): 7,
 (1, 3): 7,
 (1, 4): 7,
 (1, 5): 7,
 (0, 1, 2): 7,
 (1, 2, 3): 7,
 (1, 2, 4): 7,
 (1, 2, 5): 7,
 (0, 1, 2, 3): 7,
 (0, 1, 2, 5): 7,
 (1, 2, 3, 4): 7,
 (1, 2, 3, 5): 7,
 (0, 1, 2, 3, 4): 7,
 (1,): 8,
 (0, 2): 8,
 (2, 3): 8,
 (2, 5): 8,
 (0, 1, 3): 8,
 (0, 1, 4): 8,
 (0, 1, 5): 8,
 (0, 2, 5): 8,
 (0, 3, 4): 8,
 (0, 4, 5): 8,
 (1, 3, 4): 8,
 (2, 3, 4): 8,
 (0, 1, 3, 5): 8,
 (0, 1, 2, 3, 5): 8,
 (0, 1, 2, 4, 5): 8,
 (1, 2, 3, 4, 5): 8,
 (0,): 9,
 (0, 3): 9,
 (2, 4): 9,
 (0, 2, 3): 9,
 (0, 2, 4): 9,
 (1, 4, 5): 9,
 (2, 4, 5): 9,
 (0, 1, 2, 4): 9,
 (0, 1, 3, 4): 9,
 (0, 1, 4, 5): 9,
 (0, 2, 3, 5): 9,
 (0, 3, 4, 5): 9,
 (1, 2, 4, 5): 9,
 (2, 3, 4, 5): 9,
 (0, 1, 3, 4, 5): 9,
 (0, 1, 2, 3, 4, 5): 9,
 (0, 5): 10,
 (3, 4): 10,
 (4, 5): 10,
 (0, 3, 5): 10,
 (1, 3, 5): 10,
 (2, 3, 5): 10,
 (3, 4, 5): 10,
 (0, 2, 3, 4): 10,
 (0, 2, 4, 5): 10,
 (1, 3, 4, 5): 10,
 (0, 2, 3, 4, 5): 10,
 (3,): 12,
 (5,): 12,
 (3, 5): 12,
 (4,): 13}

In [11]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from matplotlib import image
from numpy import asarray
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def load_image(image_id,size):  
    file_path = image_id + ".jpg"
    with Image.open(IMAGE_PATH + file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

IMAGE_PATH = "./data_source/images/"
TEST_PATH = "./data_source/test.csv"
test_data = pd.read_csv(TEST_PATH)


size = (image_size[1], image_size[0])

# Test data of the challenge (no y values)
print("Loading Test Images")
X_test = []
progress = 0
for image_id in test_data["image_id"]:
    X_test.append(load_image(image_id,size) / 255.)
    print('Loading Test Images: %f%%'%((progress/len(test_data["image_id"])*100)))
    progress+=1
    clear_output(wait=True)
X_test = np.array(X_test)

Loading Test Images: 99.945085%


In [44]:
X_train = []
Y_train = []

class_map = {'healthy':0,'multiple_diseases':1,'rust':2,'scab':3}

def load_image(image_id,size):  
    file_path = image_id
    with Image.open(file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

import glob
for img_path in glob.glob("data/Test/*/*.jpg"):
    Y_train.append(class_map.get(img_path.split('\\')[1]))
    X_train.append(load_image(img_path,size) / 255.)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [61]:
def average_preds(p):
    preds = models[0].predict(p,verbose=1)
    for i in range(1,len(models)):
        preds = preds + models[i].predict(p,verbose=1)
    preds = preds / len(models)
    return preds

def ensemble(preds,mutli_prop,multi_importance = 0.8):
    for i in range(len(preds)):
        pred = preds[i]
        multi = mutli_prop[i]
        #if multi > 0.8:
        for j in range(len(pred)):
            if j==1:
                preds[i][j] = (preds[i][j]*(1-multi_importance)+(multi*multi_importance))
            else:
                preds[i][j] = (preds[i][j]*(1-multi_importance)+(1-multi)*multi_importance)
    return preds
preds = ensemble(average_preds(X_train),bin_models[0].predict(X_train),multi_importance=0)
preds_single = []
for pred in preds:
    preds_single.append(np.argmax(pred))

confusion_matrix(Y_train,preds_single)

182/182 [==============================] - 4s 25ms/step


array([[48,  0,  0,  0],
       [ 1,  8,  2,  1],
       [ 0,  1, 61,  0],
       [ 0,  1,  0, 59]], dtype=int64)

In [69]:
multi_importance = 0.575
wrongs = []
while multi_importance<=0.640000:
    preds = ensemble(average_preds(X_train),bin_models[0].predict(X_train),multi_importance=multi_importance)
    preds_single = []
    for pred in preds:
        preds_single.append(np.argmax(pred))
    wrongs.append(sum(Y_train!=preds_single))
    print('--------------')
    print('Multi_importance: %f'%multi_importance)
    print('Wrongs: %d'%sum(Y_train!=preds_single))
    print(confusion_matrix(Y_train,preds_single))
    print('--------------')
    print()
    multi_importance+=0.0026

182/182 [==============================] - 4s 25ms/step
--------------
Multi_importance: 0.510000
Wrongs: 1
[[48  0  0  0]
 [ 0 12  0  0]
 [ 0  0 62  0]
 [ 0  1  0 59]]
--------------

182/182 [==============================] - 4s 25ms/step
--------------
Multi_importance: 0.512600
Wrongs: 1
[[48  0  0  0]
 [ 0 12  0  0]
 [ 0  0 62  0]
 [ 0  1  0 59]]
--------------

182/182 [==============================] - 4s 25ms/step
--------------
Multi_importance: 0.515200
Wrongs: 1
[[48  0  0  0]
 [ 0 12  0  0]
 [ 0  0 62  0]
 [ 0  1  0 59]]
--------------

 32/182 [====>.........................] - ETA: 3s

KeyboardInterrupt: 

In [15]:
def load_image(image_id,size):  
    file_path = image_id + ".jpg"
    with Image.open(IMAGE_PATH + file_path) as image:
        image = image.resize(size)
    return np.asarray(image)


print("Loading Test Images")
X_test = []
progress = 0
for image_id in test_data["image_id"]:
    X_test.append(load_image(image_id,size) / 255.)
    print('Loading Test Images: %f%%'%((progress/len(test_data["image_id"])*100)))
    progress+=1
    clear_output(wait=True)
X_test = np.array(X_test)

Loading Test Images: 99.945085%


In [74]:
def average_preds(p,indices):
    preds = models[0].predict(p)
    preds.fill(0)
    for i in range(len(models)):
        if i in indices:
            preds = preds + models[i].predict(p)
    preds = preds / len(indices)
    return preds

def ensemble(preds,mutli_prop):
    multi_importance = 0.8
    for i in range(len(preds)):
        pred = preds[i]
        multi = mutli_prop[i]
        if multi > 0.8:
            for j in range(len(pred)):
                if j==1:
                    preds[i][j] = (preds[i][j]*(1-multi_importance)+(multi*multi_importance))
                else:
                    preds[i][j] = (preds[i][j]*(1-multi_importance)+(1-multi)*multi_importance)
    return preds

def LabelSmoothing(encodings , alpha=0.01):
    K = encodings.shape[1]
    y_ls = (1 - alpha) * encodings + alpha / K
    return y_ls

In [128]:
multi_importance = 0
#multi_importance = 0.575
probs_efnns = average_preds(X_test,[2,3])
probs_efnns = LabelSmoothing(probs_efnns)
sub.loc[:, 'healthy':] = probs_efnns
sub.to_csv('./submission_efnns.csv', index=False)
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.004976,0.004993,0.985056,0.004975
1,Test_1,0.004988,0.007106,0.982931,0.004975
2,Test_2,0.004978,0.005317,0.004975,0.984730
3,Test_3,0.985048,0.005000,0.004976,0.004976
4,Test_4,0.004982,0.017506,0.972535,0.004977
